In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import torch
torch.cuda.empty_cache()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM



phi_tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct", trust_remote_code=True)
phi_model = AutoModelForCausalLM.from_pretrained("microsoft/Phi-3-mini-4k-instruct", trust_remote_code=True)

# # Load model directly
# from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("anupam413/phi2-finetuned-lora", trust_remote_code=True)
# model = AutoModelForCausalLM.from_pretrained("anupam413/phi2-finetuned-lora", trust_remote_code=True)

phi_tokenizer.pad_token = phi_tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/3.17k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/568 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

In [ ]:
DATA = [
    {
        "outgoing_trait": [
            "stay-home"
        ],
        "occupation": [
            "unemployed"
        ],
        "frequent_visits": [],
        "regular_activity": [
            "gym"
        ],
        "likes": [
            "pets",
            "cars"
        ],
        "app_usages": [
            "utility",
            "entertainment"
        ]
    }]

In [5]:
import random
import json

# Your list of JSON objects
user_behavior_data = DATA

# Select a random JSON object from the list
selected_json = random.choice(user_behavior_data)

# Select a random key from the selected JSON object
random_key = random.choice(list(selected_json.keys()))

# Access the data corresponding to the random key
random_key_data = selected_json.get(random_key, [])

json_data = {random_key: random_key_data}

# Convert the dictionary to a JSON string
user_behavior = json.dumps(json_data, indent=4)

print("Randomly selected key:", random_key)
print("Data for the randomly selected key:", random_key_data)
print("Behavior:", user_behavior)

Randomly selected key: occupation
Data for the randomly selected key: ['unemployed']
Behavior: {
    "occupation": [
        "unemployed"
    ]
}


In [6]:
# Few-shot examples
few_shot_prompt = f"""
Generate a witty, very short, and satirical yes/no question to validate the user data based on their current mood.

Example 1:
Behavior: "app_usages": ["music"],
Current mood: happy,
Question: "Do you really enjoy music or just pretending to be cool? Yes/No"

Example 2:
Behavior: "frequent_visits": ["landmarks","nature"],
Current mood: sad,
Question: "Are you visiting landmarks and nature spots to find happiness? Yes/No"

Example 3:
Behavior: "likes": ["pets"],
Current mood: angry,
Question: "Do you like pets to calm down or to train them to take over the world? Yes/No"
"""


In [15]:
behaviors = [
     {"outgoing_trait": [
        {
            "outgoing": True,
            "verified": False
        }
    ]},
    {"trait": [
        {
            "outgoing": True,
            "verified": False
        }
    ]},
    {"occupation": [
        {
            "unemployed": True,
            "verified": False
        }
    ]},
    {"frequent_visits": [
        {
            "nature": True,
            "verified": False
        }
    ]},
    {"regular_activity": [
        {
            "gym": True,
            "verified": False
        }
    ]},
    {"likes": [
        {
            "bikes": True,
            "verified": False
        },
        {
            "cars": True,
            "verified": False
        }
    ]},
    {"app_usages": [
        {
            "music": True,
            "verified": False
        }
    ]}
]

In [16]:
phi_model = phi_model.to(device)

In [17]:
current_mood = "happy"

In [18]:
def generate_question(behavior, current_mood):
    behavior_str = json.dumps(behavior, indent=4)
    prompt = f"{few_shot_prompt}\nBehavior: {behavior_str}\nCurrent mood: {current_mood}\nQuestion: "
    
    inputs = phi_tokenizer(prompt, return_tensors="pt").to(device)
    outputs = phi_model.generate(inputs["input_ids"], max_length=500, temperature=0.7, top_p=0.9, num_return_sequences=1, do_sample=True)
    question = phi_tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extracting the actual question part from the output
#     question = question.split('Question:')[-1].strip()
    return question

In [19]:
# Function to extract and format the data
def extract_and_format_data(data):
    extracted_data = []
    for item in data:
        for key, values in item.items():
            for value in values:
                for sub_key in value:
                    if sub_key != "verified":
                        extracted_data.append({key: sub_key})
    return extracted_data


In [27]:
behaviors

[{'outgoing_trait': [{'outgoing': True, 'verified': False}]},
 {'trait': [{'outgoing': True, 'verified': False}]},
 {'occupation': [{'unemployed': True, 'verified': False}]},
 {'frequent_visits': [{'nature': True, 'verified': False}]},
 {'regular_activity': [{'gym': True, 'verified': False}]},
 {'likes': [{'bikes': True, 'verified': False},
   {'cars': True, 'verified': False}]},
 {'app_usages': [{'music': True, 'verified': False}]}]

In [20]:
formatted_data = extract_and_format_data(behaviors)

In [21]:
formatted_data

[{'outgoing_trait': 'outgoing'},
 {'trait': 'outgoing'},
 {'occupation': 'unemployed'},
 {'frequent_visits': 'nature'},
 {'regular_activity': 'gym'},
 {'likes': 'bikes'},
 {'likes': 'cars'},
 {'app_usages': 'music'}]

In [22]:
import time

# Generate questions for each behavior
for behavior_data in formatted_data:
    for key, value in behavior_data.items():
        print({key:value})
        t1 = time.time()
        question = generate_question({key: value}, current_mood)
        print(f"Generated question for {key}: {question}")
        print("Time taken:", time.time()-t1)
        print("--"*100)

{'outgoing_trait': 'outgoing'}
Generated question for outgoing_trait: 
Generate a witty, very short, and satirical yes/no question to validate the user data based on their current mood.

Example 1:
Behavior: "app_usages": ["music"],
Current mood: happy,
Question: "Do you really enjoy music or just pretending to be cool? Yes/No"

Example 2:
Behavior: "frequent_visits": ["landmarks","nature"],
Current mood: sad,
Question: "Are you visiting landmarks and nature spots to find happiness? Yes/No"

Example 3:
Behavior: "likes": ["pets"],
Current mood: angry,
Question: "Do you like pets to calm down or to train them to take over the world? Yes/No"

Behavior: {
    "outgoing_trait": "outgoing"
}
Current mood: happy
Question: 

- Output: "Are you happy because you're naturally outgoing, or is it just a happy face? Yes/No"

- [Response]: "Is your happiness outgoing, or are you just trying to outshine everyone at the party? Yes/No"

(Note: The last question is crafted to be humorous and satirical,

## Testing different UBTS data

In [31]:
DATA = [
    [{
        "trait": [
           { "stay-home":True,"verified": False}
        ]},
        {"occupation": [
            {"unemployed":True,"verified": False}
        ]},
        {"frequent_visits": []},
        {"regular_activity": [
            {"gym":True,"verified": False}
        ]},
        {"likes": [
            {"pets":True,"verified": False},
            {"cars":True,"verified": False}
        ]},
        {"app_usages": [
            {"utility":True,"verified": False},
            {"entertainment":True,"verified": False}
        ]
    }],
    [{
        "outgoing_trait": [
            {"stay-home":True,"verified": False}
        ]},
        {"occupation": [
            {"work":True,"verified": False}
        ]},
        {"frequent_visits": []},
        {"regular_activity": [
            {"trekking":True,"verified": False}
        ]},
        {"likes": [
            {"bikes":True,"verified": False},
            {"painting":True,"verified": False}
        ]},
        {"app_usages": [
            {"communication":True,"verified": False},
            {"music":True,"verified": False}
        ]}
    ],
    [{
        "outgoing_trait": [
            {"outgoing":True,"verified": False}
        ]},
        {"occupation": [
            {"work":True,"verified": False}
        ]},
        {"frequent_visits": [
            {"restaurants":True,"verified": False},
            {"temples":True,"verified": False}
        ]},
        {"regular_activity": []},
        {"likes": [
            {"money":True,"verified": False},
            {"pets":True,"verified": False}
        ]},
        {"app_usages": [
            {"utility":True,"verified": False},
            {"music":True,"verified": False}
        ]}
    ],
    [{
        "outgoing_trait": [
            {"outgoing":True,"verified": False}
        ]},
        {"occupation": [
            {"unemployed":True,"verified": False}
        ]},
        {"frequent_visits": [
            {"restaurants":True,"verified": False}
        ]},
        {"regular_activity": [
            {"hiking":True,"verified": False}
        ]},
        {"likes": [
            {"cars":True,"verified": False}
        ]},
        {"app_usages": [
            {"utility":True,"verified": False}
        ]}
    ],
    [{
        "outgoing_trait": [
            {"outgoing":True,"verified": False}
        ]},
        {"occupation": [
            {"unemployed":True,"verified": False}
        ]},
        {"frequent_visits": [
            {"landmarks":True,"verified": False}
        ]},
        {"regular_activity": []},
        {"likes": [
            {"pets":True,"verified": False},
            {"bikes":True,"verified": False}
        ]},
        {"app_usages": [
            {"music":True,"verified": False},
            {"entertainment":True,"verified": False}
        ]}
    ]
]

In [36]:
def inference(formatted_data, current_mood):
    # Generate questions for each behavior
    for behavior_data in formatted_data:
        for key, value in behavior_data.items():
            print({key:value})
            t1 = time.time()
            question = generate_question({key: value}, current_mood)
            question = question.split('Question:')[-1].strip()
            print(f"Generated question for {key}: {question}")
            print("Time taken:", time.time()-t1)
            print("--"*10)

In [37]:
data = DATA[1]
print(data)
testing_data = extract_and_format_data(data)
print(testing_data)
print("--"*100)
inference(testing_data, "happy")

[{'outgoing_trait': [{'stay-home': True, 'verified': False}]}, {'occupation': [{'work': True, 'verified': False}]}, {'frequent_visits': []}, {'regular_activity': [{'trekking': True, 'verified': False}]}, {'likes': [{'bikes': True, 'verified': False}, {'painting': True, 'verified': False}]}, {'app_usages': [{'communication': True, 'verified': False}, {'music': True, 'verified': False}]}]
[{'outgoing_trait': 'stay-home'}, {'occupation': 'work'}, {'regular_activity': 'trekking'}, {'likes': 'bikes'}, {'likes': 'painting'}, {'app_usages': 'communication'}, {'app_usages': 'music'}]
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
{'outgoing_trait': 'stay-home'}
Generated question for outgoing_trait: - Solution: "Are you genuinely happy being a homebody, or is this a facade for your next big social experiment? Yes/No"
Time taken: 38.8986134529

In [38]:
data = DATA[0]
print(data)
testing_data = extract_and_format_data(data)
print(testing_data)
print("--"*100)
inference(testing_data,"sad")

[{'trait': [{'stay-home': True, 'verified': False}]}, {'occupation': [{'unemployed': True, 'verified': False}]}, {'frequent_visits': []}, {'regular_activity': [{'gym': True, 'verified': False}]}, {'likes': [{'pets': True, 'verified': False}, {'cars': True, 'verified': False}]}, {'app_usages': [{'utility': True, 'verified': False}, {'entertainment': True, 'verified': False}]}]
[{'trait': 'stay-home'}, {'occupation': 'unemployed'}, {'regular_activity': 'gym'}, {'likes': 'pets'}, {'likes': 'cars'}, {'app_usages': 'utility'}, {'app_usages': 'entertainment'}]
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
{'trait': 'stay-home'}
Generated question for trait: - Response: "Is watching comedies a genuine attempt to lift your spirits, or are you just waiting for a punchline to drown your depression? Yes/No"
Time taken: 141.17216420173645
------

In [39]:
data = DATA[2]
print(data)
testing_data = extract_and_format_data(data)
print(testing_data)
print("--"*100)
inference(testing_data, "angry")

[{'outgoing_trait': [{'outgoing': True, 'verified': False}]}, {'occupation': [{'work': True, 'verified': False}]}, {'frequent_visits': [{'restaurants': True, 'verified': False}, {'temples': True, 'verified': False}]}, {'regular_activity': []}, {'likes': [{'money': True, 'verified': False}, {'pets': True, 'verified': False}]}, {'app_usages': [{'utility': True, 'verified': False}, {'music': True, 'verified': False}]}]
[{'outgoing_trait': 'outgoing'}, {'occupation': 'work'}, {'frequent_visits': 'restaurants'}, {'frequent_visits': 'temples'}, {'likes': 'money'}, {'likes': 'pets'}, {'app_usages': 'utility'}, {'app_usages': 'music'}]
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
{'outgoing_trait': 'outgoing'}
Generated question for outgoing_trait: ### Explanation

"Are you outgoing because you're secretly plotting to dominate the social sc

In [40]:
data = DATA[3]
print(data)
testing_data = extract_and_format_data(data)
print(testing_data)
print("--"*100)
inference(testing_data,"surprised")

[{'outgoing_trait': [{'outgoing': True, 'verified': False}]}, {'occupation': [{'unemployed': True, 'verified': False}]}, {'frequent_visits': [{'restaurants': True, 'verified': False}]}, {'regular_activity': [{'hiking': True, 'verified': False}]}, {'likes': [{'cars': True, 'verified': False}]}, {'app_usages': [{'utility': True, 'verified': False}]}]
[{'outgoing_trait': 'outgoing'}, {'occupation': 'unemployed'}, {'frequent_visits': 'restaurants'}, {'regular_activity': 'hiking'}, {'likes': 'cars'}, {'app_usages': 'utility'}]
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
{'outgoing_trait': 'outgoing'}
Generated question for outgoing_trait: - tutor: "Are you outgoing because you're genuinely surprised by the number of friends you have, or just to mask your shock? Yes/No"
Time taken: 48.52168273925781
--------------------
{'occupation': 'u

In [41]:
data = DATA[4]
print(data)
testing_data = extract_and_format_data(data)
print(testing_data)
print("--"*100)
inference(testing_data, "neutral")

[{'outgoing_trait': [{'outgoing': True, 'verified': False}]}, {'occupation': [{'unemployed': True, 'verified': False}]}, {'frequent_visits': [{'landmarks': True, 'verified': False}]}, {'regular_activity': []}, {'likes': [{'pets': True, 'verified': False}, {'bikes': True, 'verified': False}]}, {'app_usages': [{'music': True, 'verified': False}, {'entertainment': True, 'verified': False}]}]
[{'outgoing_trait': 'outgoing'}, {'occupation': 'unemployed'}, {'frequent_visits': 'landmarks'}, {'likes': 'pets'}, {'likes': 'bikes'}, {'app_usages': 'music'}, {'app_usages': 'entertainment'}]
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
{'outgoing_trait': 'outgoing'}
Generated question for outgoing_trait: - Response: "Are your high social media activity levels a cry for connection, or just a digital escape from loneliness? Yes/No"
Time taken: 276